# Import librairies

In [1]:
import geojson
import sys
import ee
import contextily

sys.path.append("..")
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings

warnings.filterwarnings("ignore")
import asyncio
import os.path
from os import path

In [2]:
import geopandas
from geopandas import GeoSeries

file = geopandas.read_file("gadm36_IND.gpkg")

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AX4XfWiDpUeHTeGMoO9uGWhTrsb8AXT869x9DU53pq1PTo8zmgRG23ub6CU



Successfully saved authorization token.


# Import data file

In [4]:
file["total_name"] = file.apply(
    lambda x: f"{x['NAME_2']}, {x['NAME_1']}, India", axis=1
)
years = np.arange(2000, 2019, 1)
years_kharif = [[f"{year}-04-16", f"{year}-10-15"] for year in years]
years_rabi = [[f"{year}-10-16", f"{year+1}-04-15"] for year in years]

In [5]:
names_to_search = file["NAME_1"].unique()

In [6]:
file.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry,total_name
0,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.1_1,Nicobar Islands,None,IND.1.1.1_1,n.a. ( 2304),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((92.78778 9.24417, 92.78889 9.2...","Nicobar Islands, Andaman and Nicobar, India"
1,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.2_1,North and Middle Andaman,None,IND.1.2.1_1,n.a. ( 2178),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((93.64841 14.93487, 93.64917 14...","North and Middle Andaman, Andaman and Nicobar,..."
2,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.3_1,South Andaman,None,IND.1.3.1_1,n.a. ( 2178),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((93.83970 12.32082, 93.85775 12...","South Andaman, Andaman and Nicobar, India"
3,IND,India,IND.2_1,Andhra Pradesh,None,IND.2.1_1,Anantapur,None,IND.2.1.1_1,Anantapur,None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((77.35452 14.52155, 77.34958 14...","Anantapur, Andhra Pradesh, India"
4,IND,India,IND.2_1,Andhra Pradesh,None,IND.2.1_1,Anantapur,None,IND.2.1.2_1,Dharmavaram,None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((77.35341 14.27068, 77.35244 14...","Anantapur, Andhra Pradesh, India"


In [7]:
file_group = file.groupby(by=["NAME_1"])["geometry"].apply(lambda x: unary_union(x))

In [8]:
file_group.loc[file_group.index[0]]

# Run import images

In [9]:
# Evap_tavg : Evapotranspiration in kg m-2 s-1
# Psurf_f_tavg : Surface pressure in Pa
# Qair_f_tavg : Specific humidity in kg kg-1
# Qs_tavg :  Storm surface runoff in kg m-2 s-1
# Qsb_tavg : Baseflow-groundwater runoff in kg m-2 s-1
# Rainf_f_tavg: Total precipitation rate in kg m-2 s-1
# SoilMoi00_10cm_tavg: Soil moisture (0 - 10 cm underground) in m^3 m-3
# SoilMoi10_40cm_tavg: Soil moisture (10 - 40 cm underground) in m^3 m-3
# SoilMoi40_100cm_tavg: Soil moisture (40 - 100 cm underground) in m^3 m-3
# SoilMoi100_200cm_tavg: Soil moisture (100 - 200 cm underground) in m^3 m-3
# SoilTemp00_10cm_tavg: Soil temperature (0 - 10 cm underground) in K
# SoilTemp10_40cm_tavg: Soil temperature (10 - 40 cm underground) in K
# SoilTemp40_100cm_tavg: Soil temperature (100 - 200 cm underground) in K
# SoilTemp100_200cm_tavg: Soil temperature (40 - 100 cm underground) in K
# Wind_f_tavg: Near surface wind speed in m s-1
bands = [
    "Evap_tavg",
    "Psurf_f_tavg",
    "Qair_f_tavg",
    "Qs_tavg",
    "Qsb_tavg",
    "Rainf_f_tavg",
    "SoilMoi00_10cm_tavg",
    "SoilMoi10_40cm_tavg",
    "SoilMoi40_100cm_tavg",
    "SoilMoi100_200cm_tavg",
    "SoilTemp00_10cm_tavg",
    "SoilTemp10_40cm_tavg",
    "SoilTemp40_100cm_tavg",
    "SoilTemp100_200cm_tavg",
    "Wind_f_tavg",
]
viz_params_list = {
    "Evap_tavg": {
        "bands": ["Evap_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0.0,
        "max": 0.00005,
        "opacity": 1.0,
        "palette": ["black", "blue", "purple", "cyan", "green", "yellow", "red"],
    },
    "Psurf_f_tavg": {
        "bands": ["Psurf_f_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 65639,
        "max": 102595,
        "opacity": 1.0,
        "palette": [
            "#01FFFF",
            "#058BFF",
            "#0600FF",
            "#DF00FF",
            "#FF00FF",
            "#FF8C00",
            "#FF8C00",
        ],
    },
    "Qair_f_tavg": {
        "bands": ["Qair_f_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0,
        "max": 0.02,
        "opacity": 1.0,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
    "Qs_tavg": {
        "bands": ["Qs_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0,
        "max": 0.0000005,
        "opacity": 1.0,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
    "Qsb_tavg": {
        "bands": ["Qsb_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0,
        "max": 0.0000003,
        "opacity": 1.0,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
    "Rainf_f_tavg": {
        "bands": ["Rainf_f_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0,
        "max": 0.00005,
        "opacity": 1.0,
        "palette": ["#FFFFFF", "#00FFFF", "#0080FF", "#DA00FF", "#FFA400", "#FF0000"],
    },
    "SoilMoi00_10cm_tavg": {
        "bands": ["SoilMoi00_10cm_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0.1,
        "max": 0.45,
        "opacity": 1.0,
        "palette": ["#FFFFFF", "#00FFFF", "#0080FF", "#DA00FF", "#FFA400", "#FF0000"],
    },
    "SoilMoi10_40cm_tavg": {
        "bands": ["SoilMoi10_40cm_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0,
        "max": 0.45,
        "opacity": 1.0,
        "palette": ["#FFFFFF", "#00FFFF", "#0080FF", "#DA00FF", "#FFA400", "#FF0000"],
    },
    "SoilMoi40_100cm_tavg": {
        "bands": ["SoilMoi40_100cm_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0.1,
        "max": 0.45,
        "opacity": 1.0,
        "palette": ["#FFFFFF", "#00FFFF", "#0080FF", "#DA00FF", "#FFA400", "#FF0000"],
    },
    "SoilMoi100_200cm_tavg": {
        "bands": ["SoilMoi100_200cm_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0.3,
        "max": 0.45,
        "opacity": 1.0,
        "palette": ["#FFFFFF", "#00FFFF", "#0080FF", "#DA00FF", "#FFA400", "#FF0000"],
    },
    "SoilTemp00_10cm_tavg": {
        "bands": ["SoilTemp00_10cm_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 280,
        "max": 310,
        "opacity": 1.0,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
    "SoilTemp10_40cm_tavg": {
        "bands": ["SoilTemp10_40cm_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 280,
        "max": 310,
        "opacity": 1.0,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
    "SoilTemp40_100cm_tavg": {
        "bands": ["SoilTemp40_100cm_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 280,
        "max": 310,
        "opacity": 1.0,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
    "SoilTemp100_200cm_tavg": {
        "bands": ["SoilTemp100_200cm_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 280,
        "max": 310,
        "opacity": 1.0,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
    "Wind_f_tavg": {
        "bands": ["Wind_f_tavg"],
        # "gain": '0.05',
        "scale": 1,
        "min": 0,
        "max": 10,
        "opacity": 1.0,
        "palette": [
            "040274",
            "040281",
            "0502a3",
            "0502b8",
            "0502ce",
            "0502e6",
            "0602ff",
            "235cb1",
            "307ef3",
            "269db1",
            "30c8e2",
            "32d3ef",
            "3be285",
            "3ff38f",
            "86e26f",
            "3ae237",
            "b5e22e",
            "d6e21f",
            "fff705",
            "ffd611",
            "ffb613",
            "ff8b13",
            "ff6e08",
            "ff500d",
            "ff0000",
            "de0101",
            "c21301",
            "a71001",
            "911003",
        ],
    },
}

In [ ]:
def inverse_coords(poly):
    x, y = poly.exterior.coords.xy
    return Polygon(zip(y, x))


errors = []
nb_images = len(file_group.index) * len(years_kharif)
# compt = 0
async def import_image(band, name, j):
    l8 = ee.ImageCollection("NASA/FLDAS/NOAH01/C/GL/M/V001").select(band)
    # compt+=1
    # print(f"Importing images is {(100*compt)//nb_images} % complete", end="\r")
    poly = file_group.loc[name]
    d_kharif = years_kharif[j]
    d_rabi = years_rabi[j]
    if path.exists(f"./moisture_data/{band}_{d_kharif}_{name}.png"):
        return
    try:
        polys = list(poly)
    except:
        polys = poly
    geopoly = GeoSeries(polys)
    try:
        geopoly = geopoly.apply(inverse_coords)
    except:
        pass
    # print(geopoly)
    geopoly = geopoly.set_crs("EPSG:4326")

    west, south, east, north = (
        geopoly.bounds["miny"].min(),
        geopoly.bounds["minx"].min(),
        geopoly.bounds["maxy"].max(),
        geopoly.bounds["maxx"].max(),
    )

    kharif = l8.filterDate(
        d_kharif[0], d_kharif[1]
    ).mosaic()  # .reduce(ee.Reducer.median())
    # .filterBounds(houston)\

    rabi = l8.filterDate(d_rabi[0], d_rabi[1]).mosaic()  # .reduce(ee.Reducer.median())
    # .filterBounds(houston)\

    viz_params = viz_params_list[band]

    try:
        # await asyncio.sleep(1)
        url_kharif = kharif.getMapId(viz_params)["tile_fetcher"].url_format
        url_rabi = rabi.getMapId(viz_params)["tile_fetcher"].url_format
        im_kharif, im_kharif_ext = contextily.bounds2img(
            west, south, east, north, ll=True, source=url_kharif
        )
        im_rabi, im_rabi_ext = contextily.bounds2img(
            west, south, east, north, ll=True, source=url_rabi
        )

        im_kharif = Image.fromarray(im_kharif)
        im_rabi = Image.fromarray(im_rabi)
        im_kharif.save(f"./moisture_data/{band}_{d_kharif}_{name}.png")
        im_rabi.save(f"./moisture_data/{band}{d_rabi}_{name}.png")
    except Exception as e:
        await asyncio.sleep(1)
        errors.append([name, j])
        print("error", e)
        # return e


async def main():
    for name in file_group.index:
        for j, _ in enumerate(years_kharif):
            for band in bands:
                await (import_image(band, name, j))


# coroutines = [import_image(name, j) for name in file_group.index for j,_ in enumerate(years_kharif)]
# await asyncio.gather(*coroutines)
import nest_asyncio

nest_asyncio.apply()
loop = asyncio.get_event_loop()
loop.run_until_complete(main())